In [2]:
import torch, json, os
from torchvision import datasets
from PIL import Image
from tqdm import tqdm
from transformers import BlipProcessor, BlipForConditionalGeneration
from diffusers import StableDiffusionPipeline
from google.colab import drive
# access drive
drive.mount('/content/drive')
blip_model_path = "/content/drive/MyDrive/blip_model"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = BlipProcessor.from_pretrained(blip_model_path, local_files_only=True)
blip_model = BlipForConditionalGeneration.from_pretrained(blip_model_path, local_files_only=True).to(device)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# cifar-10
dataset_root = "data/"
cifar = datasets.CIFAR10(root=dataset_root, train=True, download=True)

# caption
captions = {}
os.makedirs("captions", exist_ok=True)
for idx in tqdm(range(10)): #10 for testing, subject to change (200?)
    image = cifar[idx][0].resize((224, 224)).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(device)
    out = blip_model.generate(**inputs)
    captions[f"cifar10_{idx}.png"] = {"caption": processor.decode(out[0], skip_special_tokens=True)}
with open("captions/cifar10_captions.json", "w") as f:
    json.dump(captions, f, indent=2)

# gen
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4").to(device)
os.makedirs("outputs_sd/cifar10", exist_ok=True)
for name, data in captions.items():
    image = pipe(data["caption"]).images[0]
    image.save(os.path.join("outputs_sd/cifar10", name))

# regen cap
regen_captions = {}
for name in tqdm(os.listdir("outputs_sd/cifar10")):
    image = Image.open(os.path.join("outputs_sd/cifar10", name)).resize((224, 224)).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(device)
    out = blip_model.generate(**inputs)
    regen_captions[name] = {"caption": processor.decode(out[0], skip_special_tokens=True)}
os.makedirs("regenerated_captions", exist_ok=True)
with open("regenerated_captions/cifar10_sd_captions.json", "w") as f:
    json.dump(regen_captions, f, indent=2)

100%|██████████| 10/10 [00:02<00:00,  3.77it/s]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:02<00:00,  4.62it/s]

✅ Finished CIFAR-10 caption -> SD -> caption cycle
